In [23]:
import pandas as pd
from pandasql import PandaSQL
import warnings 
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [35]:
productCatalogDf = pd.read_csv('../data/product_catalog_cleaned.csv')

In [36]:
productCatalogDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Manufacturernumber       308 non-null    object 
 1   Articlenumber            308 non-null    object 
 2   EAN                      282 non-null    float64
 3   Technical details        304 non-null    object 
 4   Picture normal reduced   304 non-null    object 
 5   Depth m                  264 non-null    float64
 6   Width m                  264 non-null    float64
 7   Length m                 264 non-null    float64
 8   Weight kg                304 non-null    float64
 9   Delivery time days       4 non-null      float64
 10  Type of product          241 non-null    object 
 11  Price quantity           308 non-null    int64  
 12  ETIM Features            44 non-null     object 
 13  ETIM                     44 non-null     object 
 14  Short description        3

In [37]:
productCatalogDf.head(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Price quantity,ETIM Features,ETIM,Short description,Short description 2,Long description,Language,Manufacturername,Product_length_category,Volume_m3
0,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,1,NaN,NaN,Akku-Tauchsäge BITURBO GKT 18V-52 GC in L-BOXX,GKT 18V-52 GC (L) solo CLC,"Akku-Tauchsäge BITURBO GKT 18V-52 GC, Die Akku...",de,BOSCH,Medium,0.040599
1,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,1,NaN,NaN,Cordless plunge saw BITURBO GKT 18V-52 GC in L...,NaN,NaN,en,BOSCH,Medium,0.040599


In [38]:
productCatalogDf.tail(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Price quantity,ETIM Features,ETIM,Short description,Short description 2,Long description,Language,Manufacturername,Product_length_category,Volume_m3
306,RALB2EU,RALB2EU,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,1,§Nennspannung§18§§|§Kapazität§5§§|§Ausführung§...,EC001199,"Akku Li-Ion Bosch, 18V/5Ah",NaN,"Bosch Li-Ion Akku 18V/5Ah Akku, geeignet für d...",de,BOSCH,NaN,NaN
307,RALB2US,RALB2US,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,1,§Nennspannung§18§§|§Kapazität§5§§|§Ausführung§...,EC001199,Bosch Akku Li-Ion,NaN,"Bosch Akku 18V /5,0 Ah Li-Ion 1607A3502E",de,BOSCH,NaN,NaN


Which manufacturers have the biggest improvement potential in their data quality in absolute and relative numbers?

In [49]:
query = """
SELECT
    Manufacturername,
    COUNT(DISTINCT Articlenumber) AS Products_Count,

    -- Bad fields count per column
    SUM(CASE WHEN Manufacturernumber IS NULL THEN 1 ELSE 0 END) AS bad_Manufacturernumber,
    SUM(CASE WHEN Articlenumber IS NULL THEN 1 ELSE 0 END) AS bad_Articlenumber,
    SUM(CASE WHEN EAN IS NULL THEN 1 ELSE 0 END) AS bad_EAN,
    SUM(CASE WHEN [Technical details] IS NULL THEN 1 ELSE 0 END) AS bad_TechnicalDetails,
    SUM(CASE WHEN [Picture normal reduced] IS NULL THEN 1 ELSE 0 END) AS bad_PictureURL,
    SUM(CASE WHEN [Depth m] IS NULL THEN 1 ELSE 0 END) AS bad_Depth,
    SUM(CASE WHEN [Width m] IS NULL THEN 1 ELSE 0 END) AS bad_Width,
    SUM(CASE WHEN [Length m] IS NULL THEN 1 ELSE 0 END) AS bad_Length,
    SUM(CASE WHEN [Weight kg] IS NULL THEN 1 ELSE 0 END) AS bad_Weight,
    SUM(CASE WHEN [Delivery time days] IS NULL THEN 1 ELSE 0 END) AS bad_DeliveryTime,
    SUM(CASE WHEN [Type of product] IS NULL THEN 1 ELSE 0 END) AS bad_TypeOfProduct,
    SUM(CASE WHEN [Price quantity] IS NULL THEN 1 ELSE 0 END) AS bad_PriceQuantity,
    SUM(CASE WHEN [ETIM Features] IS NULL THEN 1 ELSE 0 END) AS bad_ETIMFeatures,
    SUM(CASE WHEN ETIM IS NULL THEN 1 ELSE 0 END) AS bad_ETIM,
    SUM(CASE WHEN [Short description] IS NULL THEN 1 ELSE 0 END) AS bad_ShortDesc,
    SUM(CASE WHEN [Short description 2] IS NULL THEN 1 ELSE 0 END) AS bad_ShortDesc2,
    SUM(CASE WHEN [Long description] IS NULL THEN 1 ELSE 0 END) AS bad_LongDesc,
    SUM(CASE WHEN Language IS NULL THEN 1 ELSE 0 END) AS bad_Language,
    SUM(CASE WHEN Manufacturername IS NULL THEN 1 ELSE 0 END) AS bad_Manufacturername,
    SUM(CASE WHEN Product_length_category IS NULL THEN 1 ELSE 0 END) AS bad_LengthCategory,
    SUM(CASE WHEN Volume_m3 IS NULL THEN 1 ELSE 0 END) AS bad_Volume,

    -- Total fields
    COUNT(*) * 21 AS Total_Records,

    -- Total Bad Fields
    (
        SUM(CASE WHEN Manufacturernumber IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Articlenumber IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN EAN IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Technical details] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Picture normal reduced] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Depth m] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Width m] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Length m] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Weight kg] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Delivery time days] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Type of product] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Price quantity] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [ETIM Features] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN ETIM IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Short description] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Short description 2] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Long description] IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Language IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Manufacturername IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Product_length_category IS NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Volume_m3 IS NULL THEN 1 ELSE 0 END)
    ) AS Total_Bad_Fields,
    
    -- Percentage of Bad Fields
    ROUND(
        (
            (
                SUM(CASE WHEN Manufacturernumber IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Articlenumber IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN EAN IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Technical details] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Picture normal reduced] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Depth m] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Width m] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Length m] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Weight kg] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Delivery time days] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Type of product] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Price quantity] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [ETIM Features] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN ETIM IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Short description] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Short description 2] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Long description] IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Language IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Manufacturername IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Product_length_category IS NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Volume_m3 IS NULL THEN 1 ELSE 0 END)
            ) * 1.0 / (COUNT(*) * 21)
        ) * 100, 1
    ) AS Percentage_Bad_Fields

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Percentage_Bad_Fields DESC;
"""

pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result.T

,0,1,2,3,4
Manufacturername,GUSTAV KLAUKE GMBH,ROTHENBERGER,FEIN,BOSCH,FISCHER
Products_Count,40,23,100,113,5
bad_Manufacturernumber,0,0,0,0,0
bad_Articlenumber,0,0,0,0,0
bad_EAN,0,0,0,26,0
bad_TechnicalDetails,0,0,0,4,0
bad_PictureURL,4,0,0,0,0
bad_Depth,40,0,0,4,0
bad_Width,40,0,0,4,0
bad_Length,40,0,0,4,0


What product variable/column (description or property) usually contains data of good quality per manufacturer? And what is the % of good quality records per variable/column and manufacturer?

In [48]:
query = """
SELECT
    Manufacturername,
    COUNT(DISTINCT Articlenumber) AS Products_Count,

    -- Good fields count per column
    SUM(CASE WHEN Manufacturernumber IS NOT NULL THEN 1 ELSE 0 END) AS good_Manufacturernumber,
    SUM(CASE WHEN Articlenumber IS NOT NULL THEN 1 ELSE 0 END) AS good_Articlenumber,
    SUM(CASE WHEN EAN IS NOT NULL THEN 1 ELSE 0 END) AS good_EAN,
    SUM(CASE WHEN [Technical details] IS NOT NULL THEN 1 ELSE 0 END) AS good_TechnicalDetails,
    SUM(CASE WHEN [Picture normal reduced] IS NOT NULL THEN 1 ELSE 0 END) AS good_PictureURL,
    SUM(CASE WHEN [Depth m] IS NOT NULL THEN 1 ELSE 0 END) AS good_Depth,
    SUM(CASE WHEN [Width m] IS NOT NULL THEN 1 ELSE 0 END) AS good_Width,
    SUM(CASE WHEN [Length m] IS NOT NULL THEN 1 ELSE 0 END) AS good_Length,
    SUM(CASE WHEN [Weight kg] IS NOT NULL THEN 1 ELSE 0 END) AS good_Weight,
    SUM(CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END) AS good_DeliveryTime,
    SUM(CASE WHEN [Type of product] IS NOT NULL THEN 1 ELSE 0 END) AS good_TypeOfProduct,
    SUM(CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) AS good_PriceQuantity,
    SUM(CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END) AS good_ETIMFeatures,
    SUM(CASE WHEN ETIM IS NOT NULL THEN 1 ELSE 0 END) AS good_ETIM,
    SUM(CASE WHEN [Short description] IS NOT NULL THEN 1 ELSE 0 END) AS good_ShortDesc,
    SUM(CASE WHEN [Short description 2] IS NOT NULL THEN 1 ELSE 0 END) AS good_ShortDesc2,
    SUM(CASE WHEN [Long description] IS NOT NULL THEN 1 ELSE 0 END) AS good_LongDesc,
    SUM(CASE WHEN Language IS NOT NULL THEN 1 ELSE 0 END) AS good_Language,
    SUM(CASE WHEN Manufacturername IS NOT NULL THEN 1 ELSE 0 END) AS good_Manufacturername,
    SUM(CASE WHEN Product_length_category IS NOT NULL THEN 1 ELSE 0 END) AS good_LengthCategory,
    SUM(CASE WHEN Volume_m3 IS NOT NULL THEN 1 ELSE 0 END) AS good_Volume,

    -- Total fields 
    COUNT(*) * 21 AS Total_Records,

    -- Total Good Fields
    (
        SUM(CASE WHEN Manufacturernumber IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Articlenumber IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN EAN IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Technical details] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Picture normal reduced] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Depth m] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Width m] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Length m] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Weight kg] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Type of product] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN ETIM IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Short description] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Short description 2] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN [Long description] IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Language IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Manufacturername IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Product_length_category IS NOT NULL THEN 1 ELSE 0 END) +
        SUM(CASE WHEN Volume_m3 IS NOT NULL THEN 1 ELSE 0 END)
    ) AS Total_Good_Fields,


    -- Percentage of Good Fields
    ROUND(
        (
            (
                SUM(CASE WHEN Manufacturernumber IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Articlenumber IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN EAN IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Technical details] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Picture normal reduced] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Depth m] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Width m] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Length m] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Weight kg] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Type of product] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN ETIM IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Short description] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Short description 2] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN [Long description] IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Language IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Manufacturername IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Product_length_category IS NOT NULL THEN 1 ELSE 0 END) +
                SUM(CASE WHEN Volume_m3 IS NOT NULL THEN 1 ELSE 0 END)
            ) * 1.0 / (COUNT(*) * 21)
        ) * 100, 1
    ) AS Percentage_Good_Fields

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Percentage_Good_Fields DESC;
"""

pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result.T

,0,1,2,3,4
Manufacturername,FISCHER,BOSCH,FEIN,ROTHENBERGER,GUSTAV KLAUKE GMBH
Products_Count,5,113,100,23,40
good_Manufacturernumber,5,140,100,23,40
good_Articlenumber,5,140,100,23,40
good_EAN,5,114,100,23,40
good_TechnicalDetails,5,136,100,23,40
good_PictureURL,5,140,100,23,36
good_Depth,5,136,100,23,0
good_Width,5,136,100,23,0
good_Length,5,136,100,23,0


Which manufacturers have the most missing or low-quality data per column?
This version gives a row per manufacturer and column-wise missing value counts and percentages.

How complete is the product data for each manufacturer across key product attributes, and which fields show the highest rates of missing or inadequate information?

In [54]:
query = """
SELECT
    Manufacturername,
    COUNT(*) AS total_products,
    COUNT(DISTINCT Articlenumber) AS Products_Count,

    -- Missing Manufacturernumber
    ROUND(SUM(CASE WHEN Manufacturernumber IS NULL OR TRIM(Manufacturernumber) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_manufacturernumber,

    -- Missing Articlenumber
    ROUND(SUM(CASE WHEN Articlenumber IS NULL OR TRIM(Articlenumber) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_articlenumber,

    -- Missing EAN
    ROUND(SUM(CASE WHEN EAN IS NULL OR TRIM(EAN) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_ean,

    -- Missing Technical details
    ROUND(SUM(CASE WHEN [Technical details] IS NULL OR TRIM([Technical details]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_technical_details,

    -- Missing Picture normal reduced
    ROUND(SUM(CASE WHEN [Picture normal reduced] IS NULL OR TRIM([Picture normal reduced]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_pictureurl,

    -- Missing Depth m
    ROUND(SUM(CASE WHEN [Depth m] IS NULL OR TRIM(CAST([Depth m] AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_depth,

    -- Missing Width m
    ROUND(SUM(CASE WHEN [Width m] IS NULL OR TRIM(CAST([Width m] AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_width,

    -- Missing Length m
    ROUND(SUM(CASE WHEN [Length m] IS NULL OR TRIM(CAST([Length m] AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_length,

    -- Missing Weight kg
    ROUND(SUM(CASE WHEN [Weight kg] IS NULL OR TRIM(CAST([Weight kg] AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_weight,

    -- Missing Delivery time days
    ROUND(SUM(CASE WHEN [Delivery time days] IS NULL OR TRIM(CAST([Delivery time days] AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_deliverytime,

    -- Missing Type of product
    ROUND(SUM(CASE WHEN [Type of product] IS NULL OR TRIM([Type of product]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_typeofproduct,

    -- Missing Price quantity
    ROUND(SUM(CASE WHEN [Price quantity] IS NULL OR TRIM([Price quantity]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_pricequantity,

    -- Missing ETIM Features
    ROUND(SUM(CASE WHEN [ETIM Features] IS NULL OR TRIM([ETIM Features]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_etimfeatures,

    -- Missing ETIM
    ROUND(SUM(CASE WHEN ETIM IS NULL OR TRIM(ETIM) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_etim,

    -- Missing Short description
    ROUND(SUM(CASE WHEN [Short description] IS NULL OR TRIM([Short description]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_shortdesc,

    -- Missing Short description 2
    ROUND(SUM(CASE WHEN [Short description 2] IS NULL OR TRIM([Short description 2]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_shortdesc2,

    -- Missing Long description
    ROUND(SUM(CASE WHEN [Long description] IS NULL OR TRIM([Long description]) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_longdesc,

    -- Missing Language
    ROUND(SUM(CASE WHEN Language IS NULL OR TRIM(Language) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_language,

    -- Missing Manufacturername
    ROUND(SUM(CASE WHEN Manufacturername IS NULL OR TRIM(Manufacturername) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_manufacturername,

    -- Missing Product_length_category
    ROUND(SUM(CASE WHEN Product_length_category IS NULL OR TRIM(Product_length_category) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_lengthcategory,

    -- Missing Volume_m3
    ROUND(SUM(CASE WHEN Volume_m3 IS NULL OR TRIM(CAST(Volume_m3 AS VARCHAR)) IN ('', 'N/A') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS pct_missing_volume

FROM productCatalogDf
GROUP BY Manufacturername
"""

result = pandasql_instance(query, locals())
result.T

,0,1,2,3,4
Manufacturername,BOSCH,FEIN,FISCHER,GUSTAV KLAUKE GMBH,ROTHENBERGER
total_products,140,100,5,40,23
Products_Count,113,100,5,40,23
pct_missing_manufacturernumber,0.0,0.0,0.0,0.0,0.0
pct_missing_articlenumber,0.0,0.0,0.0,0.0,0.0
pct_missing_ean,18.6,0.0,0.0,0.0,0.0
pct_missing_technical_details,2.9,0.0,0.0,0.0,0.0
pct_missing_pictureurl,0.0,0.0,0.0,10.0,0.0
pct_missing_depth,2.9,0.0,0.0,100.0,0.0
pct_missing_width,2.9,0.0,0.0,100.0,0.0


Completeness of Data

What is the overall completeness score of product data for each manufacturer based on key product attributes?

In [58]:
query = """
SELECT
    Manufacturername,
    ROUND(
        (
            (
                CASE WHEN Manufacturernumber IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN Articlenumber IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN EAN IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Technical details] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Picture normal reduced] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Depth m] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Width m] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Length m] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Weight kg] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Type of product] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN ETIM IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Short description] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Short description 2] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN [Long description] IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN Language IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN Manufacturername IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN Product_length_category IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN Volume_m3 IS NOT NULL THEN 1 ELSE 0 END
            ) * 100.0 / 21
        ), 1
    ) AS CompletenessPercentage

FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY CompletenessPercentage DESC;
"""

result = pandasql_instance(query, locals())
result

,Manufacturername,CompletenessPercentage
0,FISCHER,85.7
1,ROTHENBERGER,81.0
2,FEIN,81.0
3,BOSCH,81.0
4,GUSTAV KLAUKE GMBH,61.9


In [66]:
query = """
SELECT
  AVG(CompletenessScore) AS AvgCompletenessScore
FROM (
  SELECT
    (
      (CASE WHEN Manufacturernumber IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN Articlenumber IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN EAN IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Technical details] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Picture normal reduced] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Depth m] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Width m] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Length m] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Weight kg] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Delivery time days] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Type of product] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [ETIM Features] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN ETIM IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Short description] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Short description 2] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN [Long description] IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN Language IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN Manufacturername IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN Product_length_category IS NOT NULL THEN 1 ELSE 0 END) +
      (CASE WHEN Volume_m3 IS NOT NULL THEN 1 ELSE 0 END)
    ) * 1.0 / 21 AS CompletenessScore
  FROM productCatalogDf
) AS sub;
"""


result = pandasql_instance(query, locals())
result

,AvgCompletenessScore
0,0.78927


Assign a simple "data quality score" per product (out of 100)

How complete and valid is the product information for each article, measured by the percentage of key product fields that are properly filled and valid?

In [62]:
query = """
SELECT
    Articlenumber,
    Manufacturername,
    (
        (CASE WHEN [Short description] IS NOT NULL AND TRIM([Short description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Short description 2] IS NOT NULL AND TRIM([Short description 2]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Long description] IS NOT NULL AND TRIM([Long description]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Technical details] IS NOT NULL AND TRIM([Technical details]) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN EAN IS NOT NULL AND TRIM(EAN) NOT IN ('', 'N/A') THEN 1 ELSE 0 END) +
        (CASE WHEN [Picture normal reduced] IS NOT NULL AND TRIM([Picture normal reduced]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [Weight kg] IS NOT NULL AND [Weight kg] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Length m] IS NOT NULL AND [Length m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Width m] IS NOT NULL AND [Width m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Depth m] IS NOT NULL AND [Depth m] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [ETIM] IS NOT NULL AND TRIM([ETIM]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [ETIM Features] IS NOT NULL AND TRIM([ETIM Features]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [Delivery time days] IS NOT NULL AND [Delivery time days] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Type of product] IS NOT NULL AND TRIM([Type of product]) <> '' THEN 1 ELSE 0 END) +
        (CASE WHEN [Price quantity] IS NOT NULL THEN 1 ELSE 0 END) +
        (CASE WHEN [Volume_m3] IS NOT NULL AND [Volume_m3] > 0 THEN 1 ELSE 0 END) +
        (CASE WHEN [Manufacturername] IS NOT NULL AND TRIM(Manufacturername) <> '' THEN 1 ELSE 0 END)
    ) * (100.0 / 17) AS data_quality_score

FROM productCatalogDf
ORDER BY Data_Quality_Score DESC;
"""

result = pandasql_instance(query, locals())
result

,Articlenumber,Manufacturername,data_quality_score
0,06016C0000,BOSCH,82.352941
1,06012B4001,BOSCH,82.352941
2,06014A6200,BOSCH,82.352941
3,06014A6000,BOSCH,82.352941
4,06019H6L01,BOSCH,82.352941
...,...,...,...
303,ESG105RMCPCFB,GUSTAV KLAUKE GMBH,52.941176
304,LBOXXEK354CFB,GUSTAV KLAUKE GMBH,52.941176
305,LBOXXEKM60IDCFB,GUSTAV KLAUKE GMBH,52.941176
306,LS100FLEXCFB,GUSTAV KLAUKE GMBH,52.941176


Descriptive Field Interdependency

How do the presence or absence of the three key description fields (Short description, Short description 2, and Long description) interrelate across products?

In [64]:
query = """
SELECT
  CASE
    WHEN [Short description] IS NULL
         AND [Short description 2] IS NULL
         AND [Long description] IS NULL THEN 'Missing All Descriptions'

    WHEN [Short description] IS NOT NULL
         AND [Short description 2] IS NOT NULL
         AND [Long description] IS NOT NULL THEN 'Complete Descriptions'

    WHEN [Short description] IS NULL
         AND [Short description 2] IS NULL
         AND [Long description] IS NOT NULL THEN 'Only Long Description Present'

    WHEN [Short description] IS NULL
         AND [Short description 2] IS NOT NULL
         AND [Long description] IS NULL THEN 'Only Short Description 2 Present'

    WHEN [Short description] IS NOT NULL
         AND [Short description 2] IS NULL
         AND [Long description] IS NULL THEN 'Only Short Description Present'

    WHEN [Short description] IS NOT NULL
         AND [Short description 2] IS NULL
         AND [Long description] IS NOT NULL THEN 'Short Description + Long Present'

    WHEN [Short description] IS NULL
         AND [Short description 2] IS NOT NULL
         AND [Long description] IS NOT NULL THEN 'Short Description 2 + Long Present'

    ELSE 'Other / Mixed'
  END AS description_completeness_combo,
  COUNT(*) AS product_count
FROM productCatalogDf
GROUP BY description_completeness_combo
ORDER BY product_count DESC;
"""

result = pandasql_instance(query, locals())
result

,description_completeness_combo,product_count
0,Short Description + Long Present,168
1,Complete Descriptions,109
2,Other / Mixed,25
3,Short Description 2 + Long Present,2
4,Only Short Description Present,2
5,Only Short Description 2 Present,1
6,Only Long Description Present,1
